<a href="https://colab.research.google.com/github/JosenildoVicente/Algoritmo-de-agrupamento-adaptavel-baseado-em-kNN-e-densidade/blob/main/ACND.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
import pandas as pd
import numpy as np

from google.colab import files
import io
import numpy.ma as ma
from scipy.spatial import distance as dist


In [80]:
data = pd.read_csv( "https://raw.githubusercontent.com/JosenildoVicente/Algoritmo-de-agrupamento-adaptavel-baseado-em-kNN-e-densidade/main/Aggregation.csv", encoding="ISO-8859-1" )

In [81]:
data.head()

,Col_0,Col_1
0,15.55,28.65
1,14.90,27.55
2,14.45,28.35
3,14.15,28.80
4,13.75,28.05


# Etapa 1: Read and analyze data, and determine parameters

In [82]:
N = data.shape[0]
alfa = 0.0
alpha = 0.0
beta = 0.0
gama = 0.0


In [83]:
for m in data:

  max_s = max(data[m])
  min_s = min(data[m])
  rs = max_s - min_s
  alfa += rs

# α = N/alpha; β = (3.5α + 45)/100;
#γ = (2α + 18)/100; k = max(10, N∗1%);

alpha = N/alfa
beta = ((3.5 * alpha) + 45) / 100
gama = ( (2 * alpha) + 18)/100
k = max(10, (N * 0.01))
print(k)

10


#Etapa 2: Construct k-D Tree: kDT

In [84]:
from sklearn.neighbors import KDTree
kdt = KDTree(data)
print(kdt.data.shape)

(788, 2)


In [ ]:
kdt.get_arrays()

#Etapa 3: Searches kNN of sample, calculates mean of 5NN’s

In [86]:
distance = []
knn_index = []
d5nn = []

distance, knn_index = kdt.query(data,k)

for i in range(len(knn_index)):
  d5nn.append(np.mean(distance[i, 1:6]))


#Etapa 4: Calculate the global radius of SR

In [87]:
d5nn = np.sort(d5nn)

In [88]:
R = max(d5nn)
R

1.8010955367628583

#Etapa 5: Calculate sample density array(Rho)

In [89]:
def fab(i, j, a, b):
  d = dist.euclidean(i, j)
  if d >= 0 and d <= a:
    return 1
  elif d > a and d <= b:
    return ((b - d)/ (b-a))
  elif d > b:
    return 0

In [143]:
rho = []
for i in range(N):
  rho.append(0)
for i in range(N):
  surrounding_region = distance[i,] < R
  surrounding_region = distance[i,surrounding_region]
  surrounding_region = surrounding_region[1:]

  mean_ = np.mean(surrounding_region)
  std_dev = np.std(surrounding_region)
  surrounding_region = np.sort(surrounding_region)
  d1 = 0.1 * surrounding_region
  d2 = min(surrounding_region)
  a = (d1 + d2)/2
  exp = (a + 2*std_dev)
  b = min( min(exp), R)

#   for (j in surrounding-region) {
# f(i, j) = fab(i, j, a, b) // Eq. (5)
# Rho[i] + = f(i, j);}//for j}//for i
  # rho.append(0)
  for j in surrounding_region:#range(len(surrounding_region)):
    f = fab(distance[i,1],j,min(a),b)
    rho[i] += f
  print(i,"  a: ", a,"  b: ",b,"  rho:",rho[i])



0   a:  [0.62709648 0.63397237 0.64048835]   b:  0.8458110961549413   rho: 3
1   a:  [0.50483289 0.52163867 0.52282366 0.53148208 0.53182589 0.54249287
 0.54855624]   b:  1.0404179919373093   rho: 5.728159922693369
2   a:  [0.29745798 0.30849521 0.31631025 0.32564315 0.32742512 0.33603837
 0.35321882]   b:  0.9761863076720334   rho: 4.347104050846974
3   a:  [0.29745798 0.31291635 0.31407697 0.33052042 0.34081698 0.34330324
 0.34424046]   b:  0.9735522934718246   rho: 3.6901318669623446
4   a:  [0.31112698 0.31416762 0.32092158 0.32534271 0.34184483 0.34554239
 0.34941035 0.35107943 0.35742488]   b:  0.9673199992809531   rho: 5.589868549211392
5   a:  [0.31112698 0.3219739  0.32650334 0.33060515 0.3354022  0.33806952
 0.33902323 0.3412058  0.34433458]   b:  0.711790015939844   rho: 5.578647407918805
6   a:  [0.43044309 0.4384818  0.44260761 0.45141597 0.45787953 0.462901
 0.47411437 0.47682505]   b:  1.0551295702643726   rho: 5.955486510379969
7   a:  [0.34457401 0.34660326 0.36101154 

#Etapa 6: Calculate the global density threshold(T)

In [105]:
# for (i = 0; i < N; i + + ) Arr[i] = (Rho[i], i);
# sort(Arr); // in descending order
# T = lower-Quartile(Arr);
arr = []
for i in range(N):
  arr.append((rho[i], i))

arr = np.sort(arr,0)
arr = arr[::-1]

t = np.percentile(arr, 25) 
t

5.366751577237822

#Etapa 7: Clustering

In [158]:
order_of_cluster = 0
tag = []
for i in range(N):
  tag.append(False)
queue = []

result = []
for y in range(N):
  result.append(0)

# print(R)

for ind in range(N):
  # 1 representa indice e 0 valor de arr
  # print(np.int(arr[ind][1]))
  if ( tag[np.int(arr[ind][1])]):
    continue
  if (arr[ind][0] == 0):
    tag[np.int(arr[ind][1])] = True
    continue
  if ( arr[ind][0] < t ):
    continue

  order_of_cluster += 1
  tag[ np.int(arr[ind][1]) ] = True
  queue.clear()
  queue.append(arr[ind][1])
  # queue.pop(len(queue)-1)
  # print(queue)

  while (len(queue) > 0):
    # print(queue)
    i = queue.pop(0)
    # print(len(queue))
    aver = np.mean( rho )
    local_t = aver * beta
    # print(local_t)
    for e in range(k):
      j = knn_index[np.int(i)][e]

      if (distance[j][e] > R):
        continue
      
      co_nn = 0
      for a in range(1,10):
        for b in range(1,10):
          if (knn_index[np.int(i)][a] == knn_index[j][b]):
            co_nn += 1
      # print(co_nn)
      tag[j] = True
      print(co_nn > (k * gama))
      if (rho[j]):
        result[j] = order_of_cluster
        if (rho[j] >= local_t and co_nn > (k * gama)):   #(Rho[j] > = localT && co_NN > k ∗γ )
          queue.append(j)
          continue



A saída de streaming foi truncada nas últimas 5000 linhas.
True
True
True
True
True
True
True
True
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
False
False
True
False
False
True
True
True
True
True
True
True
False
True
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
True
True
True
True
True
True
False
True
False
False
True
True
True
True
True
True
True
True
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
True
False
False
True
True
True
True
True
True
True
False
False
False
True
True
True
False
False
True
False
False
True
False
True
True
True
True
True
True
False
True
False
False
True
True
True
True
False
False
True
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
False
False
True
False
False
True
False
True
True
True
True
True
True
Tr

KeyboardInterrupt: ignored